In [3]:
# 데이터, 종목명 사전 불러오기
import pandas as pd

news_df = pd.read_csv('/Users/JooAnLee/final_project/db/news_2023_2025.csv')
kospi_df = pd.read_csv('/Users/JooAnLee/final_project/db/KRX_KOSPI.csv', encoding='cp949')

In [4]:
news_df.info()
print('-----------------------------------------------------')
kospi_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58405 entries, 0 to 58404
Data columns (total 7 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   news_id  58405 non-null  object
 1   wdate    58405 non-null  object
 2   title    58405 non-null  object
 3   article  58405 non-null  object
 4   press    58405 non-null  object
 5   url      58405 non-null  object
 6   image    58405 non-null  object
dtypes: object(7)
memory usage: 3.1+ MB
-----------------------------------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 845 entries, 0 to 844
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   종목코드    845 non-null    int64  
 1   종목명     845 non-null    object 
 2   종가      845 non-null    int64  
 3   대비      845 non-null    int64  
 4   등락률     845 non-null    float64
 5   상장시가총액  845 non-null    float64
dtypes: float64(2), int64(3), object(1)
memory usage: 39.7+ KB


In [5]:
# 종목명만 set으로 (사전에 NaN 제거)
company_names = set(kospi_df['종목명'].dropna().unique())

In [6]:
# 제목 + 본문 결합
news_df['text_combined'] = news_df['title'].fillna('') + ' ' + news_df['article'].fillna('')
news_df['text_combined'] = news_df['text_combined'].str.replace(r'\s+', ' ', regex=True).str.strip()

In [7]:
import random

# 랜덤 500개 샘플 추출
sample_size = 500
random_indices = random.sample(range(len(news_df)), min(sample_size, len(news_df)))
sample_df = news_df.iloc[random_indices].copy()

# 각 뉴스에 대해 매칭된 종목명 찾기
def find_matching_companies(text, company_names):
    """텍스트에서 종목명 사전과 매칭되는 기업명들을 찾는 함수"""
    if pd.isna(text):
        return []
    
    matched_companies = []
    text_lower = text.lower()  # 대소문자 구분 없이 매칭
    
    for company in company_names:
        if company.lower() in text_lower:
            matched_companies.append(company)
    
    return matched_companies

# 매칭 실행
sample_df['matched_companies'] = sample_df['text_combined'].apply(
    lambda x: find_matching_companies(x, company_names)
)

# 매칭된 종목 수 계산
sample_df['num_matched'] = sample_df['matched_companies'].apply(len)

# 결과 확인
print("=== 매칭 결과 요약 ===")
print(f"전체 샘플 수: {len(sample_df)}")
print(f"매칭된 뉴스 수: {len(sample_df[sample_df['num_matched'] > 0])}")
print(f"매칭 비율: {len(sample_df[sample_df['num_matched'] > 0]) / len(sample_df) * 100:.1f}%")
print(f"평균 매칭 종목 수: {sample_df['num_matched'].mean():.2f}")

# 매칭된 예시 몇 개 확인
print("\n=== 매칭된 뉴스 예시 ===")
matched_samples = sample_df[sample_df['num_matched'] > 0].head(5)

for idx, row in matched_samples.iterrows():
    print(f"\n[뉴스 {idx}]")
    print(f"제목: {row['title'][:100]}...")
    print(f"매칭된 종목: {row['matched_companies']}")
    print(f"매칭 수: {row['num_matched']}")


=== 매칭 결과 요약 ===
전체 샘플 수: 500
매칭된 뉴스 수: 419
매칭 비율: 83.8%
평균 매칭 종목 수: 2.67

=== 매칭된 뉴스 예시 ===

[뉴스 16746]
제목: [단독]'콜옵션' 해소한 DN솔루션즈…다음달 IPO 출격...
매칭된 종목: ['대상', 'DN오토모티브']
매칭 수: 2

[뉴스 17685]
제목: [클릭 e종목]"LG화확, 내년 이익 반등 유효"...
매칭된 종목: ['LG', '전방', 'NH투자증권', 'LG화학', 'LG에너지솔루션']
매칭 수: 5

[뉴스 53451]
제목: 카카오게임즈, 신작 흥행 여부가 관건…목표주가 2.6만원-KB...
매칭된 종목: ['카카오']
매칭 수: 1

[뉴스 26131]
제목: 다시 '5만전자' 된 삼성전자, 쉽지 않을 반등...
매칭된 종목: ['삼성전자', 'NH투자증권', '유진투자증권', '현대차', 'DB', '현대차증권']
매칭 수: 6

[뉴스 55862]
제목: 추락하는 홍콩증시… ELS 이어 ETN도 대규모 손실 우려...
매칭된 종목: ['LS', '대상', '삼성증권', '미래에셋증권']
매칭 수: 4


1. BERT 멀티라벨 분류 모델 직접 학습
2. 미리 학습된 문장 임베딩(예: SBERT) → 유사도 점수
각 기사와 종목명(설명)을 임베딩한 뒤,
코사인 유사도(0~1점)를 점수로 사용
예: “삼성전자” 관련 기사와 “삼성전자” 임베딩 유사도 = 0.91
훈련 없이 “빠른 점수화” 가능
실전에서는 “멀티라벨 BERT 분류”가 가장 많이 쓰임

In [8]:
sample_df.head()

,news_id,wdate,title,article,press,url,image,text_combined,matched_companies,num_matched
16746,20241227_0113,2024-12-27 10:17,[단독]'콜옵션' 해소한 DN솔루션즈…다음달 IPO 출격,해넘겨 상장…콜옵션 부담 해소\n상장 시 기업가치 5조~6조 기대\n이 기사는 12...,한국경제,https://n.news.naver.com/mnews/article/015/000...,https://imgnews.pstatic.net/image/015/2024/12/...,[단독]'콜옵션' 해소한 DN솔루션즈…다음달 IPO 출격 해넘겨 상장…콜옵션 부담 ...,"[대상, DN오토모티브]",2
23646,20241031_0260,2024-10-31 09:13,"포커스에이치엔에스, 20억 규모 자사주 취득 신탁계약 “주주가치 제고”",포커스에이치엔에스 제공\n[파이낸셜뉴스] 포커스에이치엔에스가 주가 안정과 주주가치 ...,파이낸셜뉴스,https://n.news.naver.com/mnews/article/014/000...,https://imgnews.pstatic.net/image/014/2024/10/...,"포커스에이치엔에스, 20억 규모 자사주 취득 신탁계약 “주주가치 제고” 포커스에이치...",[],0
17685,20241218_0257,2024-12-18 07:45,"[클릭 e종목]""LG화확, 내년 이익 반등 유효""",목표주가 기존 대비 14% 하향 조정\nNH투자증권은 18일 LG화학에 대해 전방 ...,아시아경제,https://n.news.naver.com/mnews/article/277/000...,https://imgnews.pstatic.net/image/277/2024/12/...,"[클릭 e종목]""LG화확, 내년 이익 반등 유효"" 목표주가 기존 대비 14% 하향 ...","[LG, 전방, NH투자증권, LG화학, LG에너지솔루션]",5
53451,20240213_0323,2024-02-13 08:09,"카카오게임즈, 신작 흥행 여부가 관건…목표주가 2.6만원-KB",경기 성남 분당구 카카오게임즈 본사/사진=뉴스1 제공\nKB증권이 카카오게임즈에 대...,머니투데이,https://n.news.naver.com/mnews/article/008/000...,https://imgnews.pstatic.net/image/008/2024/02/...,"카카오게임즈, 신작 흥행 여부가 관건…목표주가 2.6만원-KB 경기 성남 분당구 카...",[카카오],1
51252,20240229_0155,2024-02-29 11:12,"폴라리스오피스, 온디바이스 AI 오피스 상용화 소식에 22%대↑","특징주\n/사진제공=폴라리스오피스, 뉴시스\n폴라리스오피스가 인터넷 없이 사용할 수...",머니투데이,https://n.news.naver.com/mnews/article/008/000...,https://imgnews.pstatic.net/image/008/2024/02/...,"폴라리스오피스, 온디바이스 AI 오피스 상용화 소식에 22%대↑ 특징주 /사진제공=...",[],0


## snunlp/KR-FinBERT 모델로 종목명 추출된 80% 확률값 알아보기

In [10]:
# 1. 회사명 라벨 인덱스 맵핑
all_company_names = sorted(list(company_names))
company2id = {c: i for i, c in enumerate(all_company_names)}
id2company = {i: c for c, i in company2id.items()}

def labels_to_vec(label_list, company2id):
    vec = [0]*len(company2id)
    for c in label_list:
        if c in company2id:
            vec[company2id[c]] = 1
    return vec

sample_df["label_vec"] = sample_df["matched_companies"].apply(lambda x: labels_to_vec(x, company2id))

In [11]:
# 2. 토크나이저, 데이터셋 준비
from transformers import AutoTokenizer

model_name = "snunlp/KR-FinBERT"  # or "kaist-ai/KF-Deberta-v1-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)

from torch.utils.data import Dataset

class NewsDataset(Dataset):
    def __init__(self, df, tokenizer, max_len=512):
        self.texts = df["text_combined"].tolist()
        self.labels = df["label_vec"].tolist()
        self.tokenizer = tokenizer
        self.max_len = max_len
    def __len__(self):
        return len(self.texts)
    def __getitem__(self, idx):
        item = self.tokenizer(self.texts[idx], truncation=True, padding="max_length", max_length=self.max_len, return_tensors='pt')
        item = {k: v.squeeze() for k, v in item.items()}
        item["labels"] = torch.tensor(self.labels[idx], dtype=torch.float)
        return item

train_dataset = NewsDataset(sample_df, tokenizer)

In [12]:
# 3. 모델 선언
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=len(company2id),
    problem_type="multi_label_classification"
)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at snunlp/KR-FinBERT and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# 4. 훈련 및 추론
import torch
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./output",
    num_train_epochs=5,
    per_device_train_batch_size=4,
    logging_steps=5,
    learning_rate=2e-5,
    # evaluation_strategy="no",  # <- 이 줄 지움!
    # save_strategy="no",       # <- 이 줄도 지움!
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
)

trainer.train()

/opt/anaconda3/envs/final_jn/lib/python3.9/site-packages/torch/utils/data/dataloader.py:683: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Step,Training Loss
5,0.682300


In [ ]:
#5. 추론 예측 및 실험
test_text = sample_df["text_combined"].iloc[0]
inputs = tokenizer(test_text, return_tensors="pt", truncation=True, padding=True, max_length=512)
import torch
with torch.no_grad():
    logits = model(**inputs).logits
    probs = torch.sigmoid(logits).numpy().flatten()
    top5_idx = probs.argsort()[-5:][::-1]
    print("Top 5 관련 종목:")
    for i in top5_idx:
        print(f"{id2company[i]}: {probs[i]:.2f}")